<a href="https://colab.research.google.com/github/Marin-kh/Persian_RAG/blob/main/Persian_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hazm
!pip install python-docx
!pip install rake_nltk
!pip install docx
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53892 sha256=0d39074495644ae68830fece689b5261ef4bb90c65c702863216001701b0bca3
  Stored in directory: /root/.cache/pip/wheels/c1/3e/c3/e81c11effd0be5658a035947c66792dd993bcff317eae0e1ed
Successfully built docx
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [2]:
import numpy as np
import pandas as pd
from hazm import stopwords_list, Normalizer, WordTokenizer, SentenceTokenizer, Stemmer, Lemmatizer, sent_tokenize, word_tokenize
import docx
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import requests
from sklearn.metrics.pairwise import cosine_similarity
import openai
import nltk
from rake_nltk import Rake
from google.colab import drive
import stanza
from collections import defaultdict

In [3]:
stanza.download('fa')
nlp = stanza.Pipeline('fa')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: fa (Persian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/fa/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: fa (Persian):
| Processor | Package        |
------------------------------
| tokenize  | perdt          |
| mwt       | perdt          |
| pos       | perdt_charlm   |
| lemma     | perdt_nocharlm |
| depparse  | perdt_charlm   |
| ner       | arman          |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [4]:
class PersianRAKE(Rake):
    def _tokenize_text_to_sentences(self, text: str):
        return sent_tokenize(text)

    def _tokenize_sentence_to_words(self, sentence: str):
        return word_tokenize(sentence)

In [5]:
def read_from_docx(doc):
    fullText=''
    for pra in doc.paragraphs:
        fullText+=pra.text+' '

    return fullText

def split_into_overlapping_chunks(sentences, max_chunk_size=1000, overlap_size=200):
    chunks = []
    current_chunk = ""
    current_chunk_size = 0

    for sentence in sentences:
        sentence_length = len(sentence)

        if current_chunk_size + sentence_length > max_chunk_size and current_chunk:
            chunks.append(current_chunk.strip())

            overlap_buffer = current_chunk[-overlap_size:].strip() if current_chunk else ""
            current_chunk = overlap_buffer + " "
            current_chunk_size = len(overlap_buffer) + 1

        current_chunk += sentence + " "
        current_chunk_size += sentence_length + 1

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

def preprocess_text_1(text):
    # text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'( +)', ' ', str(text))
    return text


def preprocess_text_2(text):
    text = re.sub('(\(.*?\))|(\[.*?\])', '', str(text))
    text = re.sub(r'( +)', ' ', str(text))

    word_tokenizer = WordTokenizer()
    words = word_tokenizer.tokenize(text)

    stopwords = stopwords_list()
    filtered_words = [word for word in words if word not in stopwords]

    lemmatizer = Lemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    return ' '.join(lemmatized_words)


def check_spelling(main_text):
    endpoint = "https://api.languagetool.org/v2/check"

    data = {
        "text": main_text,
        "language": "en-US",
    }

    response = requests.post(endpoint, data=data)
    json_response = response.json()

    updated_text = main_text

    for match in json_response.get("matches", []):
        replacement = match["replacements"][0]["value"] if match["replacements"] else ""

        offset = match["offset"]
        length = match["length"]

        updated_text = updated_text.replace(main_text[offset:offset+length], replacement)

    print("Original Query: ", main_text)
    print("Spell-checked Query: ", updated_text)
    return updated_text

def phrase_search(sentence):
    doc = nlp(sentence)

    phrases = []
    for sent in doc.sentences:
        for word in sent.words:
            if word.upos in ['NOUN', 'ADJ']:
                phrase = word.text
                for other_word in sent.words:
                    if other_word.head == word.id and other_word.upos in ['NOUN', 'ADJ']:
                        phrase += " " + other_word.text
                if " " in phrase:
                    phrases.append(phrase)
    return phrases

def english_to_persian_number(number_str):
    english_to_persian = {
        "0": "۰",
        "1": "۱",
        "2": "۲",
        "3": "۳",
        "4": "۴",
        "5": "۵",
        "6": "۶",
        "7": "۷",
        "8": "۸",
        "9": "۹",
    }
    persian_number = "".join([english_to_persian[digit] for digit in number_str])
    return persian_number

def persian_words_to_number(sentence):
    word_to_number = {
        "صفر": 0,
        "یک": 1,
        "دو": 2,
        "سه": 3,
        "چهار": 4,
        "پنج": 5,
        "شش": 6,
        "هفت": 7,
        "هشت": 8,
        "نه": 9,
        "ده": 10,
        "یازده": 11,
        "دوازده": 12,
        "سیزده": 13,
        "چهارده": 14,
        "پانزده": 15,
        "شانزده": 16,
        "هفده": 17,
        "هجده": 18,
        "نوزده": 19,
        "بیست": 20,
        "سی": 30,
        "چهل": 40,
        "پنجاه": 50,
        "شصت": 60,
        "هفتاد": 70,
        "هشتاد": 80,
        "نود": 90,
        "صد": 100,
        "یکصد": 100,
        "دویست": 200,
        "سیصد": 300,
        "چهارصد": 400,
        "پانصد": 500,
        "ششصد": 600,
        "هفتصد": 700,
        "هشتصد": 800,
        "نهصد": 900,
        "هزار": 1000,
    }
    words = sentence.split(' ')

    result = []
    temp_number_words = []
    current_number = 0

    for word in words:
        if word[-1:]=='م' and (word[:-1] in word_to_number):
            word = word[:-1]
        if word in word_to_number:
            temp_number_words.append(word)
            current_number += word_to_number[word]
        else:
            if temp_number_words:
                english_number_str = str(current_number)
                persian_number_str = english_to_persian_number(english_number_str)
                result.append(persian_number_str)
                temp_number_words = []
                current_number = 0
            result.append(word)

    if temp_number_words:
        english_number_str = str(current_number)
        persian_number_str = english_to_persian_number(english_number_str)
        result.append(persian_number_str)

    return ' '.join(result)

def preprocess_phrases(text, phrases):
    for phrase in phrases:
        text = text.replace(phrase, phrase.replace(" ", "_"))
    return text

def extract_persian_numbers(text):
    persian_digits = "۰۱۲۳۴۵۶۷۸۹"
    return re.findall(f"[{persian_digits}]+", text)

def calculate_cosine_similarity(docs, phrase):
    vectorizer = TfidfVectorizer(ngram_range=(1, 3))
    tfidf_matrix = vectorizer.fit_transform(docs)
    phrase_vector = vectorizer.transform(phrase)
    return cosine_similarity(phrase_vector, tfidf_matrix)

def calculate_tf(document_numbers):
    tf = []
    for doc in document_numbers:
        tf_dict = defaultdict(int)
        for num in doc:
            tf_dict[num] += 1
        tf.append(tf_dict)
    return tf

def calculate_idf(document_numbers, numbers):
    idf = {}
    total_docs = len(document_numbers)
    for num in numbers:
        doc_count = sum(1 for doc in document_numbers if num in doc)
        idf[num] = np.log((total_docs + 1) / (doc_count + 1)) + 1
    return idf

def calculate_tf_idf(document_numbers, numbers):
    tf = calculate_tf(document_numbers)
    idf = calculate_idf(document_numbers, numbers)
    tf_idf = []
    for doc_tf in tf:
        doc_tf_idf = {}
        for num, freq in doc_tf.items():
            if num in idf:
                doc_tf_idf[num] = freq * idf[num]
        tf_idf.append(doc_tf_idf)
    return tf_idf

In [6]:
# Loading The Main Document
drive.mount('/content/drive')
document = read_from_docx(docx.Document("/content/drive/My Drive/Constitution_of_the_Islamic_Republic.docx"))

Mounted at /content/drive


In [7]:
# Chunking The Document
normalizer = Normalizer()
normalized_text = normalizer.normalize(document)

sentence_tokenizer = SentenceTokenizer()
sentences = sentence_tokenizer.tokenize(normalized_text)

max_chunk_size = 1000
overlap_size = 200
chunks = split_into_overlapping_chunks(sentences, max_chunk_size, overlap_size)
print("<Chunk 1>")
print(f"Original Chunk:\n{chunks[0]}")

# Preprocessing The Chunks
preprocessed1_chunks = [preprocess_text_1(chunk) for chunk in chunks]

preprocessed2_chunks = [preprocess_text_2(chunk) for chunk in preprocessed1_chunks]
print(f"Preprocessed Chunk:\n{preprocessed2_chunks[0]}")

<Chunk 1>
Original Chunk:
قانون اساسی جمهوری اسلامی ایران مبین نهادهای فرهنگی، اجتماعی، سیاسی و اقتصادی جامعه ایران بر اساس اصول و ضوابط اسلامی است که انعکاس خواست قلبی امت اسلامی می‏باشد. ماهیت انقلاب عظیم اسلامی ایران و روند مبارزه مردم مسلمان از ابتدا تا پیروزی که در شعارهای قاطع و کوبنده همه قشرهای مردم تبلور می‏یافت این خواست اساسی را مشخص کرده و اکنون در طلیعه این پیروزی بزرگ، ملت ما با تمام وجود نیل به آن را می‏طلبد. ویژگی بنیادی این انقلاب نسبت به دیگر نهضت‏های ایران در سده اخیر، مکتبی و اسلامی بودن آن است. ملت مسلمان ایران پس از گذر از نهضت ضد استبدادی مشروطه و نهضت ضد استعماری ملی‏ شدن نفت به این تجربه گرانبار دست یافت که علت اساسی و مشخص عدم موفقیت این نهضت‏ها مکتبی نبودن مبارزات بوده است. گرچه در نهضت‏های اخیر خط فکری اسلامی و رهبری روحانیت مبارز سهم اصلی و اساسی را بر عهده داشت، ولی به دلیل دور شدن این مبارزات از مواضع اصیل اسلامی، جنبش‏ها به سرعت به رکود کشانده شد.
Preprocessed Chunk:
قانون اساسی جمهوری اسلامی ایران مبین نهاد فرهیخت#فرهنگ اجتماعی سیاسی اقتصادی جامعه ایران

In [8]:
query = "اصل هفتم قانون اساسی جمهوری اسلامی درباره چیست؟"
query = check_spelling(query)

processed_query = preprocess_text_2(preprocess_text_1(query))
print(query, "\n", processed_query)

Original Query:  اصل هفتم قانون اساسی جمهوری اسلامی درباره چیست؟
Spell-checked Query:  اصل هفتم قانون اساسی جمهوری اسلامی درباره چیست؟
اصل هفتم قانون اساسی جمهوری اسلامی درباره چیست؟ 
 اصل هفتم قانون اساسی جمهوری اسلامی چیست


In [9]:
query_n = persian_words_to_number(processed_query)
print(query_n)

اصل ۷ قانون اساسی جمهوری اسلامی چیست


In [10]:
phrases = phrase_search(query_n)
print(phrases)

['اصل ۷', '۷ قانون', 'قانون اساسی']


In [12]:
numbers = re.findall(r'\d+', query_n)

print(numbers)

['۷']


In [13]:
numberic_chunks = [persian_words_to_number(chunk) for chunk in preprocessed2_chunks]

In [14]:
# Similarity Of Query
query_bonus = calculate_cosine_similarity(numberic_chunks, [query_n]).flatten()


# Similarity Of Numbers
document_numbers = [extract_persian_numbers(doc) for doc in numberic_chunks]
tf_idf_numbers = calculate_tf_idf(document_numbers, numbers)

number_bonus = np.zeros(len(numberic_chunks))
for i, doc_tf_idf in enumerate(tf_idf_numbers):
    for num in numbers:
        if num in doc_tf_idf:
            number_bonus[i] += doc_tf_idf[num]
if np.max(number_bonus) > 0:
    number_bonus = number_bonus / np.max(number_bonus)


# Similarity Of Phrases
preprocessed_docs = [preprocess_phrases(doc, phrases) for doc in numberic_chunks]
preprocessed_phrases = preprocess_phrases(query_n, phrases)

phrases_similarity = calculate_cosine_similarity(preprocessed_docs, [preprocessed_phrases])
phrases_bonus = np.max(phrases_similarity, axis=0)
if np.max(phrases_bonus) > 0:
    phrases_bonus = phrases_bonus / np.max(phrases_bonus)


query_coef = 0.5
numbers_coef = 0.3
phrases_coef = 0.2

hybrid_scores = (query_bonus * query_coef) + (number_bonus * numbers_coef) + (phrases_bonus * phrases_coef)

In [15]:
top_k = 3
indices = np.argsort(-hybrid_scores)[:top_k]

top_3 = [chunks[idx] for idx in indices]

print("Top 3 Hybrid Results:")
for idx, chunk in zip(indices, top_3):
    print(f"Chunk {idx + 1} (Score: {hybrid_scores[idx]:.2f}):\n{chunk}")

Top 3 Hybrid Results:
Chunk 65 (Score: 0.45):
دالت و تقوای لازم برای رهبری امت اسلام ۳ - بینش صحیح سیاسی و اجتماعی، تدبیر، شجاعت، مدیریت و قدرت کافی برای رهبری. در صورت تعدد واجدین شرایط فوق، شخصی که دارای بینش فقهی و سیاسی قوی‏تر باشد مقدم است. اصل یکصد و دهم وظایف و اختیارات رهبر: ۱ - تعیین سیاست‏های کلی نظام جمهوری اسلامی ایران پس از مشورت با مجمع تشخیص مصلحت نظام ۲ - نظارت بر حسن اجرای سیاست‏های کلی نظام ۳ - فرمان همه‏پرسی ۴ - فرماندهی کل نیروهای مسلح ۵ - اعلان جنگ و صلح و بسیج نیروها ۶ - نصب و عزل و قبول استعفا‏ء: الف- فقهای شورای نگهبان ب- عالی‏ترین مقام قوه قضائیه ج- رئیس سازمان صدا و سیمای جمهوری اسلامی ایران د- رئیس ستاد مشترک ه‍- فرمانده کل سپاه پاسداران انقلاب اسلامی و- فرماندهان عالی نیروهای نظامی و انتظامی ۷ - حل اختلاف و تنظیم روابط قوای سه‏گانه ۸ - حل معضلات نظام که از طرق عادی قابل‌حل نیست، از طریق مجمع تشخیص مصلحت نظام ۹ - امضاء حکم ریاست جمهوری پس از انتخاب مردم- صلاحیت داوطلبان ریاست جمهوری از جهت دارا بودن شرایطی که در این قانون می‏آید، باید قبل از انتخابات به تأیید

In [16]:
df = pd.DataFrame()
df = pd.read_csv('/content/drive/My Drive/api_key.csv')

In [22]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=df.loc[0, 'api_key'],
)

completion = client.chat.completions.create(
  extra_headers={
  },
  model="GPT-4o-mini",
  messages=[
    {
      "role": "user",
      "content": f"{top_3[0]}\n{top_3[1]}\n{top_3[2]}\nطبق متن های بالا به طور خلاصه(در حد یک پاراگراف) به این سوال جواب بده و اشاره ای به کلمه پاراگراف نکن:{query}\n"
    }
  ],
  temperature=0.2
)
print(completion.choices[0].message.content)

اصل هفتم قانون اساسی جمهوری اسلامی ایران به اهمیت شوراها در تصمیم‌گیری و اداره امور کشور اشاره دارد. طبق این اصل، شوراها شامل مجلس شورای اسلامی، شورای استان، شهرستان، شهر و دیگر سطوح، ارکان اصلی تصمیم‌گیری هستند و وظیفه دارند که بر اساس مشورت و همکاری با یکدیگر، امور کشور را مدیریت کنند. همچنین، شرایط و حدود اختیارات این شوراها توسط قانون مشخص می‌شود و بر اساس دستور قرآن کریم، مشورت و همکاری میان مردم و دولت از اصول اساسی نظام است.
